<a href="https://colab.research.google.com/github/diagomike/Amharic_cleaners/blob/master/registeryourcontactsinitiative.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@markdown ## <font color="pink"> *****EWCA Zone 1 Ethiopia `.VCF` MASS REGISTRATION***** 📝
#@markdown ##<font color="YELLOW"> NOW ITS EASY TO MAKE SURE YOU ***REACH ALL YOUR CONTACTS***
import requests
from bs4 import BeautifulSoup
from google.colab import files
from requests import Session
import json
import os
import quopri

def decode_quoted_printable(encoded_str):
    try:
        return quopri.decodestring(encoded_str).decode('utf-8','replace').replace('�',"")
    except:
        quopri.decodestring(encoded_str.encode('latin-1')).decode('utf-8')

def clean_phone_number(phone:str):
    # Remove non-numeric characters
    if phone.startswith("+"):
        cleaned_phone = "+" + ''.join(filter(str.isdigit, phone))
    else:
        cleaned_phone = ''.join(filter(str.isdigit, phone))

    # Standardize phone number format
    if len(cleaned_phone) == 10 and cleaned_phone.startswith('0'):  # Remove leading 0 OF ETHIOPIAN NUMBERS
        cleaned_phone = "+251" + cleaned_phone[1:]
    elif len(cleaned_phone) == 11 and cleaned_phone.startswith('0'):  # Remove leading 0 OF NIGERIAN NUMBERS
        cleaned_phone = "+234" + cleaned_phone[1:]
    # ELSE WE CANNOT DECIDE ACCURATELY THE COUNTRY CODE
    # for your own country code add logic here, or just use international format contacts
    elif not cleaned_phone.startswith("+"):
        cleaned_phone = None

    return cleaned_phone

def extract_contacts_from_vcf(vcf_file):
    contacts = []
    invalid_nums = []
    with open(vcf_file, 'r',encoding='utf8') as f:
        vcf_data = f.read()

    vcard_segments = vcf_data.split('BEGIN:VCARD')[1:]

    for segment in vcard_segments:
        # print(segment)
        lines = segment.split('\n')
        firstname = None
        phone = None
        invalid = False
        for line in lines:
            if line.startswith('FN'):
                firstname = line.split(':')[-1]
                if line.__contains__('CHARSET=UTF-8'):
                    firstname = decode_quoted_printable(firstname)
            elif line.startswith('TEL'):
                phone_s = line.split(':')[-1]
                phone = clean_phone_number(phone=phone_s)
                if not phone:
                    invalid = True

        if firstname and phone:
            # Name Cleaning!!!
            fn_parts = firstname.split()
            if len(fn_parts) == 1:
                firstname = 'Highly_Esteemed'
                lastname = fn_parts[0]
            elif len(fn_parts) == 2:
                firstname, lastname = fn_parts
            else:
                firstname =fn_parts[-2]
                lastname = fn_parts[-1]
            try:
              contacts.append((firstname,lastname,phone))
            except:
              #choosen charset handle ignore (local unbound error lastname)
              invalid_nums.append((firstname,firstname,phone_s))
        elif invalid:
            invalid_nums.append((firstname,lastname,phone_s))

    return contacts,invalid_nums

def extract_contacts_from_txt(txt_file):
    with open(txt_file, 'rb') as file:
        content = file.read().decode('utf16', errors='replace').replace('�',"")
    data = [info.split('\t') for info in content.splitlines()]

    contacts = []
    invalid_nums = []
    for firstname,phone in data:
        if phone.startswith("+") or \
           (phone.startswith("251") and len(phone) == 12) or \
           (phone.startswith("243") and len(phone) == 13) or \
            (len(phone) == 9 and phone.isdigit() and phone.startswith(('7','9'))) or \
            (len(phone) == 10 and phone.isdigit() and phone.startswith(('80','90','70','81'))):
            if firstname and phone:
                # Name Cleaning!!!
                fn_parts = firstname.strip().split()
                if len(fn_parts) == 1:
                    firstname = 'Highly_Esteemed'
                    lastname = fn_parts[0]
                elif len(fn_parts) == 2:
                    firstname, lastname = fn_parts
                else:
                    firstname =fn_parts[-2]
                    lastname = fn_parts[-1]
                if phone:
                    if not phone.startswith(('80','90','70','81')):
                      phone = phone if not phone.startswith(("251","243")) else phone[3:] # 251 & 243
                      phone = phone if phone.startswith("+") else f"+251{phone}"
                    else:
                      phone = phone if phone.startswith("+") else f"+234{phone}"
                    contacts.append((firstname,lastname,phone))
        else:
            invalid_nums.append((firstname,firstname,phone))

    return contacts,invalid_nums

reg_url = "https://healingstreams.tv/LHS/new_phone_action.php"
confirm_url = "https://healingstreams.tv/LHS/online_reg.php?r=EWCA1"


headers = {
    "cookie": "auth_ref=EWCA1; PHPSESSID=eb92c462a4c160a68be286149002c676; _gcl_au=1.1.773488742.1708589760; _gid=GA1.2.633684381.1708589761; record_added=NO; auth_indicate=1; auth_phone=251948618139; auth_firstname=Daniel; auth_lastname=Mitiku; HAPPIDS1=s3|Zdd0V|ZddgD; _ga_VZX8DZVY9D=GS1.1.1708618409.5.1.1708618836.0.0.0; _uetsid=9d89c8b0d15a11eea40989666bbcd335; _uetvid=776e4a50562311ee86d7c1a9d252a2ca; _ga_EWYM605EQB=GS1.1.1708618409.5.1.1708618837.58.0.0; _ga=GA1.2.1594716718.1708589761; _gat_gtag_UA_174949206_1=1",
    "authority": "healingstreams.tv",
    "accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7",
    "accept-language": "en-US,en;q=0.9",
    "cache-control": "max-age=0",
    "content-type": "application/x-www-form-urlencoded",
    "origin": "https://healingstreams.tv",
    "referer": "https://healingstreams.tv/LHS/online_reg.php?r=EWCA1",
    "sec-ch-ua": '"Not A(Brand";v="99", "Microsoft Edge";v="121", "Chromium";v="121"',
    "sec-ch-ua-mobile": "?0",
    "sec-ch-ua-platform": '"Windows"',
    "sec-fetch-dest": "document",
    "sec-fetch-mode": "navigate",
    "sec-fetch-site": "same-origin",
    "sec-fetch-user": "?1",
    "upgrade-insecure-requests": "1",
    "user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/121.0.0.0 Safari/537.36 Edg/121.0.0.0"
}

def register_new(fname:str,lname:str,phone:str,s:requests.Session)-> str:
    #destroying Generalization :: Province is Oromia, City is Adama
    # state = state_map[state_name]
                                                    # ignoring + sign
    payload = f"fullname={fname}%20{lname}&phone1={phone}&zone={zone}&country1={country}&state1={state}&city2={city}&phone_reg=Submit"

    ighealing = True #Don't fill Checkboxes
    igpartner = True #Don't fill Checkboxes
    payload = payload if ighealing else payload + '&need_healing2=on'
    payload = payload if igpartner else payload + '&partner=on'

    response = s.request("POST", reg_url, headers=headers, data=payload,params=querystring)

    # Check if the request was successful
    if response.status_code == 200:
        # Parse the HTML content
        soup = BeautifulSoup(response.content, 'html.parser')

        # Find the confrimation
        try:
            name = soup.select("body > div.wrapper > div > div > div.col-lg-7.col-md-7 > h4")[0].text.split(' for registering')[0][10:]
            return f"{fname} {lname}" == name # Only Works if Registered!
        except:
            return False
        # return if Confrimed!

def confirm_reg(phone:str,fname:str,lname:str,s:requests.Session)-> str:
                            # ignoring + sign
    payload = f"my_details={phone}&email_check=Submit%20Email%2FPhone"
    response = s.request("POST", confirm_url, data=payload, headers=headers, params=querystring)

    # Check if the request was successful
    if response.status_code == 200:
        # Parse the HTML content
        soup = BeautifulSoup(response.content, 'html.parser')

        # Find the name
        print(f"{fname} {lname} with +{phone} ",end=' ')
        try :
            name = soup.select("#registernow > div > div.col-lg-7.col-md-7 > h6 > div > h3")[0].text.split(' for')[0][12:]
            print("is confrimed!")
            return name
        except:
            try :
                # document.querySelector("#registernow > div > div.col-lg-7.col-md-7 > h2 > font:nth-child(1)")
                soup.select("#widget > h6 > font")[0].text

                if register_new(fname,lname,phone,s):
                    print("New Person Registered!")
                    return "New"
                else:
                    print("Number Not Accepted at Register!")
                    return 'Error Connect'
            except:
                print("Number Not Accepted at Confirm!")
                return "Invalid Number"
    else:
       print("Skipped for Connection!")
       return 'Error Connect'


def confirm_self(phone:str):
    phone = ''.join(filter(str.isdigit, phone))
    payload = f"my_details={phone}&email_check=Submit%20Email%2FPhone"
    response = requests.request("POST", confirm_url, data=payload, headers=headers, params=querystring)

    # Check if the request was successful
    if response.status_code == 200:
        # Parse the HTML content
        soup = BeautifulSoup(response.content, 'html.parser')

        # Find the name
        try :
            name = soup.select("#registernow > div > div.col-lg-7.col-md-7 > h6 > div > h3")[0].text.split(' for')[0][12:]
            return name
        except:
            print('please register yourself manually first!')

def load_progress(file:str)->dict:
    # Check if the file exists
    if os.path.isfile(f'{file}.json'):
        # Open the file and load the data
        print("from loaded past progress!")
        with open(f'{file}.json', 'r', encoding='utf8') as f:
            return json.load(f)
    else:
        with open(f'{file}.json', 'w', encoding='utf8') as f:
            json.dump({},f)
        print("from Scratch!")
        return {}

def update_progress(file:str,report:dict):
    data = load_progress(file=file)
    data.update(report)
    save_progress(file=file,report=data)
    print('progress Updated')

def save_progress(file:str,report:dict):
    # If Latter we deal with 100k + numbers
    with open(f'{file}.json', 'w', encoding='utf8') as f:
        json.dump(report,f)

def get_names(data:tuple[list,list],file_path:str)->dict:
    report = load_progress(file=file_path)
    for fname,lname,phone in data[0]: # data[1] are invalid_nums
        name = f'{fname} {lname}' #for Record Keeping
        phone = ''.join(filter(str.isdigit, phone)) # removing the + sign ahead
        session = Session()

        # only retry for invalid phone numbers
        if name in report and not (report[name][0] == 'Invalid Number!' or report[name][0] == None):
            continue

        reg_name = confirm_reg(fname=fname,lname=lname,phone=phone,s=session)
        if reg_name == "New":
            report[name] = 'New Registered!',phone  # New Person 🎉 Excitement overload! 🎉
        elif reg_name == "Invalid Number":
            report[name] = 'Invalid Number!',phone
        elif reg_name != 'Error Connect':
            report[name] = reg_name,phone # Confirming Previously Registered Person

        save_progress(file=file_path,report=report)


    # Saving Readable final Report!
    final_report = '\uFEFFName,Phone,Status\n'
    total =  0
    for name,(reg,phone) in report.items():
        if reg == 'New Registered!':total = total + 1
        final_report += name+','+phone+','+reg+'\n'
    # Show the Invalid numbers at the bottom
    for fname,lname,phone in data[1]:
        final_report += f"{fname} {lname}, {str(phone)} ,Invalid Number\n"
    final_report += f'New, Registered, {total}'
    open(f'registration_report.csv', 'w', encoding='utf8').write(final_report)
    print(f'all Info saved in registration_report.csv')
    return report

################################################################################
################################################################################
###### CONSTACTS FOR ETHIOPIA {IF REGISTERING ANOTHER ZONE & COUNTRY CHANGE HERE!}
################################################################################
################################################################################
zone = 'EWCA%20Zone%201' #EWCA Zone 1
country = '70' #Ethiopia

region = 'Oromia Region' #@param ['Addis Ababa','Afar Region','Amhara Region','Benishangul-Gumuz Region','Dire Dawa','Gambela Region','Harari Region','Oromia Region','Somali Region','Southern Nations, Nationalities, and Peoples Region','Tigray Region']
city = 'Adama' #@param {'type':'string'}
ChurchLink = 'CEADAMA' #@param {'type':'string'}
#@markdown final registration url: https://healingstreams.tv/zone/{ChruchLink}
querystring = {"r": ChurchLink}
state_map =  {'Addis Ababa': '11','Afar Region': '6',
              'Amhara Region': '3','Benishangul-Gumuz Region': '9',
              'Dire Dawa': '8','Gambela Region': '10',
              'Harari Region': '7', 'Oromia Region': '5',
              'Somali Region': '2', 'Tigray Region': '4',
              "Southern Nations, Nationalities, and Peoples Region": '1'}
state = state_map[region]

YOUR_NUMBER_INTERNATIONAL = '+251975072718' #@param {'type':'string'}
print("\n\033[34m\033[1mUPLOAD YOUR PHONE_BOOK(list)...")
listfn, length = files.upload().popitem()

if listfn != 'contacts.vcf':
  !mv "$listfn" contacts.vcf

if listfn.endswith('.vcf'):
    extracted_info, invalid_nums = extract_contacts_from_vcf("contacts.vcf")
elif listfn.endswith('.txt'):
    extracted_info, invalid_nums = extract_contacts_from_txt("contacts.vcf")


print(f"Extracted {len(extracted_info)} contacts🤩 Heaven is Rejoicing...👏👏👏")

name = confirm_self(phone=YOUR_NUMBER_INTERNATIONAL)
if name:
  print(f'Welcome to Mass Registration {name}')
  f_name,l_name = name.split(' ')

  #putting Credit where Credit is due!
  headers['cookie'] = "auth_ref=EWCA1; PHPSESSID=eb92c462a4c160a68be286149002c676; _gcl_au=1.1.773488742.1708589760; _gid=GA1.2.633684381.1708589761; record_added=NO; auth_indicate=1; auth_phone=251{Your_phone}; auth_firstname={f_name}; auth_lastname={l_name}; HAPPIDS1=s3|Zdd0V|ZddgD; _ga_VZX8DZVY9D=GS1.1.1708618409.5.1.1708618836.0.0.0; _uetsid=9d89c8b0d15a11eea40989666bbcd335; _uetvid=776e4a50562311ee86d7c1a9d252a2ca; _ga_EWYM605EQB=GS1.1.1708618409.5.1.1708618837.58.0.0; _ga=GA1.2.1594716718.1708589761; _gat_gtag_UA_174949206_1=1"
  print('Starting Confirmations',end=' ')
  get_names(data=(extracted_info,invalid_nums),file_path='data.csv')


files.download('registration_report.csv')